In [ ]:
import arviz as az
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.handlers import seed
from numpyro.infer import MCMC, NUTS, SVI, Predictive, Trace_ELBO
from numpyro.infer.autoguide import AutoDelta, AutoNormal
from scipy.interpolate import InterpolatedUnivariateSpline

from stream_membership import ComponentMixtureModel, ModelComponent
from stream_membership.distributions import IndependentGMM, TruncatedNormalSpline
from stream_membership.plot import plot_data_projections

## Simulate data

First we generate some simulated data to have a similar complexity to real stream data.

In [ ]:
phi1_lim = (-100, 20)
phi2_lim = (-8, 4)
pm1_lim = (None, -2.0)

In [ ]:
N_data = 8192

# first, the background:
keys = jax.random.split(jax.random.PRNGKey(42), num=9)
N = int(N_data * 0.55)
data = {
    "phi1": dist.TruncatedNormal(
        loc=-70, scale=30, low=phi1_lim[0], high=phi1_lim[1]
    ).sample(keys[1], (N,)),
    "phi2": dist.Uniform(*phi2_lim).sample(keys[2], (N,)),
}
true_pm1_loc_spl = InterpolatedUnivariateSpline(
    [-100, -20, 0.0, 20], [-5.0, -1.0, 2.0, 5.0]
)
true_pm1_scale = 2.0
data["pm1"] = dist.TruncatedNormal(
    loc=true_pm1_loc_spl(data["phi1"]), scale=2, high=pm1_lim[1]
).sample(keys[3])

# now, add a "stream":
keys = jax.random.split(jax.random.PRNGKey(412), num=6)
N_stream = int(0.85 * (N_data - N))
stream_data = {}
true_phi1_loc = -20.0
true_phi1_scale = 40.0
stream_data["phi1"] = dist.TruncatedNormal(
    loc=-20, scale=40, low=phi1_lim[0], high=phi1_lim[1]
).sample(keys[1], (N_stream,))

true_phi2_loc_spl = InterpolatedUnivariateSpline(
    [-100, -60, -20, 0, 20], [-6, -2, 0, 0, -1], k=3
)
true_phi2_ln_scale_spl = InterpolatedUnivariateSpline(
    [-100, -60, -20, 0, 20], np.log([0.8, 0.6, 0.5, 0.5, 0.8]), k=3
)
stream_data["phi2"] = TruncatedNormalSpline(
    loc_vals=[-6, -2, 0, 0, -1],
    scale_vals=[0.8, 0.6, 0.5, 0.5, 0.8],
    knots=[-100, -60, -20, 0, 20],
    low=phi2_lim[0],
    high=phi2_lim[1],
    x=stream_data["phi1"],
).sample(keys[2])

true_pm1_loc_spl = InterpolatedUnivariateSpline(
    [-100, -60, -20, 0, 20], [-3, -6, -9, -6, -4], k=3
)
true_pm1_ln_scale = np.log(0.05)
stream_data["pm1"] = TruncatedNormalSpline(
    loc_vals=[-3, -6, -9, -6, -4],
    scale_vals=np.full(5, 0.05),
    knots=[-100, -60, -20, 0, 20],
    low=pm1_lim[0],
    high=pm1_lim[1],
    x=stream_data["phi1"],
).sample(keys[3])

# Optionally: add an "off-track" feature
N_offrack = N_data - N - N_stream
offtrack = {}
offtrack["phi1"] = dist.TruncatedNormal(
    loc=-30, scale=6, low=phi1_lim[0], high=phi1_lim[1]
).sample(keys[4], (N_offrack,))

offtrack["phi2"] = TruncatedNormalSpline(
    loc_vals=np.array([-6.0, -2.0, 0.5, 2, 2.5, 2.5]),
    scale_vals=np.full(6, 0.3),
    knots=[-100, -60, -40, -20, 0, 20],
    low=phi2_lim[0],
    high=phi2_lim[1],
    x=offtrack["phi1"],
).sample(keys[5])

offtrack["pm1"] = TruncatedNormalSpline(
    loc_vals=[-3, -6, -9, -6, -4],
    scale_vals=np.full(5, 0.05),
    knots=[-100, -60, -20, 0, 20],
    low=pm1_lim[0],
    high=pm1_lim[1],
    x=offtrack["phi1"],
).sample(keys[6])

data = {k: jnp.concatenate([data[k], stream_data[k], offtrack[k]]) for k in data}
print(N_data, N, N_stream, N_offrack)

pm1_err = jnp.exp(jax.random.uniform(keys[7], minval=-5, maxval=0, shape=(N_data,)))
data["pm1"] = jax.random.normal(keys[8], shape=(N_data,)) * pm1_err + data["pm1"]
err = {"pm1": pm1_err}
# data = stream_data

In [ ]:
plot_grids = {
    "phi1": jnp.linspace(*phi1_lim, 512),
    "phi2": jnp.linspace(*phi2_lim, 128),
    "pm1": jnp.linspace(-15, pm1_lim[1], 128),
}
fig, axes = plot_data_projections(data, grids=plot_grids)
fig.suptitle("simulated data", fontsize=24)

### Define the background model

The sky distribution is represented by a grid of 2D Normal distributions. The grid Normal components have different learned scales for the $\phi_1$ and $\phi_2$ directions. The proper motion track is a truncated Normal with a mean and log-stddev that are spline functions of $\phi_1$.

In [ ]:
bkg_phi12_locs = jnp.stack(
    jnp.meshgrid(
        jnp.arange(phi1_lim[0] - 20, phi1_lim[1] + 20 + 1e-3, 20),
        jnp.arange(phi2_lim[0] - 4, phi2_lim[1] + 4 + 1e-3, 4),
    ),
    axis=-1,
).reshape(-1, 2)
bkg_pm1_knots = jnp.arange(-100, 20 + 1e-3, 30)

bkg_model = ModelComponent(
    name="background",
    coord_distributions={
        "phi1": dist.TruncatedNormal,
        "phi2": dist.Uniform,
        "pm1": TruncatedNormalSpline,
    },
    coord_parameters={
        "phi1": {
            "loc": dist.Normal(-70.0, 50.0),
            "scale": dist.HalfNormal(50.0),
            "low": phi1_lim[0],
            "high": phi1_lim[1],
        },
        "phi2": {
            "low": phi2_lim[0],
            "high": phi2_lim[1],
        },
        "pm1": {
            "loc_vals": dist.Normal(pm1_lim[1], 1.0).expand([bkg_pm1_knots.shape[0]]),
            "scale_vals": dist.HalfNormal(5).expand([bkg_pm1_knots.shape[0]]),
            "knots": bkg_pm1_knots,
            "x": data["phi1"],
            "low": pm1_lim[0],
            "high": pm1_lim[1],
            "spline_k": 3,
            "clip_locs": (-10, pm1_lim[1]),
        },
    },
    conditional_data={"pm1": {"x": "phi1"}},
)

In [ ]:
tmp = np.exp(-0.5 * (bkg_phi12_locs[:, 0] - -80.0) ** 2 / 20**2)
bkg_init_params = {
    # ("phi1", "phi2"): {
    #     "mixing_distribution": tmp / tmp.sum(),
    #     "scales": jnp.array([20.0, 4.0])[:, None],
    # },
    "phi1": {"loc": -70.0, "scale": 30.0},
    "phi2": {},
    "pm1": {
        "loc_vals": jnp.full(bkg_pm1_knots.shape[0], -5.0),
        "scale_vals": jnp.full(bkg_pm1_knots.shape[0], 10.0),
    },
}

The stream model is defined using a 1D mixture of Normal distributions for the $\phi_1$ density, and conditional Normal distributions for $\phi_2$ and $\mu_1$ tracks. 

In [ ]:
stream_phi1_knots = jnp.arange(-100, 20 + 1e-3, 10)
stream_phi2_knots = jnp.arange(-100, 20 + 1e-3, 15)
stream_pm1_knots = jnp.arange(-100, 20 + 1e-3, 15)

pm1_knots = jnp.arange(-100, 20 + 1e-3, 15)
stream_model = ModelComponent(
    name="stream",
    coord_distributions={
        "phi1": IndependentGMM,
        "phi2": TruncatedNormalSpline,
        "pm1": TruncatedNormalSpline,
    },
    coord_parameters={
        "phi1": {
            "mixing_distribution": (
                dist.Categorical,
                dist.Dirichlet(jnp.ones(len(stream_phi1_knots))),
            ),
            "locs": stream_phi1_knots.reshape(1, -1),
            "scales": dist.HalfNormal(5.0),
            "low": jnp.array([phi1_lim[0]])[:, None],
            "high": jnp.array([phi1_lim[1]])[:, None],
        },
        "phi2": {
            "loc_vals": dist.Uniform(*phi2_lim).expand([stream_phi2_knots.shape[0]]),
            # "scale_vals": dist.Uniform(-6, 1).expand([stream_phi2_knots.shape[0]]),
            "scale_vals": dist.HalfNormal(1.0).expand([stream_phi2_knots.shape[0]]),
            "knots": stream_phi2_knots,
            "x": data["phi1"],
            "low": phi2_lim[0],
            "high": phi2_lim[1],
            "spline_k": 3,
        },
        "pm1": {
            "loc_vals": dist.Uniform(-15, pm1_lim[1]).expand(
                [stream_pm1_knots.shape[0]]
            ),
            # "ln_scale_vals": dist.Uniform(-6, 1),
            # "scale_vals": dist.HalfNormal(1.0),
            "scale_vals": dist.Uniform(0.1, 1.0),
            "knots": stream_pm1_knots,
            "x": data["phi1"],
            "low": pm1_lim[0],
            "high": pm1_lim[1],
            "spline_k": 3,
            "clip_locs": (-10.0, pm1_lim[1]),
        },
    },
    conditional_data={"phi2": {"x": "phi1"}, "pm1": {"x": "phi1"}},
)
stream_init_params = {
    "phi1": {
        "locs": jnp.ones(len(stream_phi1_knots)) / len(stream_phi1_knots),
        "scales": 10.0,
    },
    "phi2": {
        "loc_vals": jnp.full(stream_phi2_knots.shape[0], 0.0),
        "scale_vals": jnp.full(stream_phi2_knots.shape[0], 0.5),
    },
    "pm1": {
        "loc_vals": jnp.full(stream_pm1_knots.shape[0], -5.0),
        "scale_vals": 0.1,
    },
}

We also include an "off-track" model that uses the 2D grid of Normals for the sky distribution but uses the same $\mu_1$ track as the stream model. Because we will

In [ ]:
offtrack_phi12_locs_sparse = jnp.stack(
    jnp.meshgrid(
        jnp.arange(phi1_lim[0], phi1_lim[1] + 1e-3, 4),
        jnp.arange(phi2_lim[0], phi2_lim[1] + 1e-3, 1.0),
    ),
    axis=-1,
).reshape(-1, 2)
offtrack_phi12_locs_dense = jnp.stack(
    jnp.meshgrid(
        jnp.arange(-50.0, -10.0 + 1e-3, 1),
        jnp.arange(-2.0, 3.0 + 1e-3, 0.25),
    ),
    axis=-1,
).reshape(-1, 2)
offtrack_phi12_locs = jnp.concatenate(
    (offtrack_phi12_locs_sparse, offtrack_phi12_locs_dense), axis=0
)
jnp.unique(offtrack_phi12_locs, axis=0).shape, offtrack_phi12_locs.shape

In [ ]:
# offtrack_phi12_locs = jnp.stack(
#     jnp.meshgrid(
#         jnp.arange(phi1_lim[0], phi1_lim[1] + 1e-3, 4),
#         jnp.arange(phi2_lim[0], phi2_lim[1] + 1e-3, 0.5),
#     ),
#     axis=-1,
# ).reshape(-1, 2)

# offtrack_model = ModelComponent(
#     name="offtrack",
#     coord_distributions={
#         ("phi1", "phi2"): IndependentGMM,
#         "pm1": stream_model.coord_distributions["pm1"],
#     },
#     coord_parameters={
#         ("phi1", "phi2"): {
#             "mixing_distribution": (
#                 dist.Categorical,
#                 dist.Dirichlet(jnp.ones(offtrack_phi12_locs.shape[0])),
#             ),
#             "locs": offtrack_phi12_locs.T,
#             # "scales": jnp.array([[5.0, 1.0]]).T,
#             # "scales": dist.HalfNormal(jnp.array([5., 1.])[:, None]),
#             "scales": dist.Uniform(0.5, 10.0).expand((2, 1)),
#             "low": jnp.array([phi1_lim[0], phi2_lim[0]])[:, None],
#             "high": jnp.array([phi1_lim[1], phi2_lim[1]])[:, None],
#         },
#         # We don't need to define the parameters here, as they are the same as for the
#         # stream model - this will be handled below when we combine the components into
#         # a mixture model
#         "pm1": stream_model.coord_parameters["pm1"],
#     },
#     conditional_data={"pm1": {"x": "phi1"}},
# )
# offtrack_init_params = {
#     ("phi1", "phi2"): {
#         "mixing_distribution": jnp.ones(offtrack_phi12_locs.shape[0])
#         / offtrack_phi12_locs.shape[0],
#         "scales": jnp.array([5.0, 1.0])[:, None],
#     }
# }

In [ ]:
offtrack_phi12_locs_sparse = jnp.stack(
    jnp.meshgrid(
        jnp.arange(phi1_lim[0], phi1_lim[1] + 1e-3, 4),
        jnp.arange(phi2_lim[0], phi2_lim[1] + 1e-3, 1.0),
    ),
    axis=-1,
).reshape(-1, 2)
offtrack_phi12_locs_dense = jnp.stack(
    jnp.meshgrid(
        jnp.arange(-50.0, -10.0 + 1e-3, 1),
        jnp.arange(-2.0, 3.0 + 1e-3, 0.25),
    ),
    axis=-1,
).reshape(-1, 2)

offtrack_model_dense = ModelComponent(
    name="offtrack_dense",
    coord_distributions={
        ("phi1", "phi2"): IndependentGMM,
        "pm1": stream_model.coord_distributions["pm1"],
    },
    coord_parameters={
        ("phi1", "phi2"): {
            "mixing_distribution": (
                dist.Categorical,
                dist.Dirichlet(jnp.full(offtrack_phi12_locs_dense.shape[0], 0.5)),
            ),
            "locs": offtrack_phi12_locs_dense.T,
            "scales": dist.Uniform(0.2, 10.0).expand((2, 1)),
            "low": jnp.array([phi1_lim[0], phi2_lim[0]])[:, None],
            "high": jnp.array([phi1_lim[1], phi2_lim[1]])[:, None],
        },
        # We don't need to define the parameters here, as they are the same as for the
        # stream model - this will be handled below when we combine the components into
        # a mixture model
        "pm1": stream_model.coord_parameters["pm1"],
    },
    conditional_data={"pm1": {"x": "phi1"}},
)
offtrack_init_params_dense = {
    ("phi1", "phi2"): {
        "mixing_distribution": jnp.ones(offtrack_phi12_locs_dense.shape[0])
        / offtrack_phi12_locs_dense.shape[0],
        "scales": jnp.array([1.0, 0.25])[:, None],
    }
}

offtrack_model_sparse = ModelComponent(
    name="offtrack_sparse",
    coord_distributions={
        ("phi1", "phi2"): IndependentGMM,
        "pm1": stream_model.coord_distributions["pm1"],
    },
    coord_parameters={
        ("phi1", "phi2"): {
            "mixing_distribution": (
                dist.Categorical,
                dist.Dirichlet(jnp.full(offtrack_phi12_locs_sparse.shape[0], 4.0)),
            ),
            "locs": offtrack_phi12_locs_sparse.T,
            "scales": dist.Uniform(1.0, 10.0).expand((2, 1)),
            "low": jnp.array([phi1_lim[0], phi2_lim[0]])[:, None],
            "high": jnp.array([phi1_lim[1], phi2_lim[1]])[:, None],
        },
        # We don't need to define the parameters here, as they are the same as for the
        # stream model - this will be handled below when we combine the components into
        # a mixture model
        "pm1": stream_model.coord_parameters["pm1"],
    },
    conditional_data={"pm1": {"x": "phi1"}},
)
offtrack_init_params_sparse = {
    ("phi1", "phi2"): {
        "mixing_distribution": jnp.ones(offtrack_phi12_locs_sparse.shape[0])
        / offtrack_phi12_locs_sparse.shape[0],
        "scales": jnp.array([4.0, 1.0])[:, None],
    }
}

Now that we have defined the three components of the model, we combine them into a mixture model. The mixture model is a mixture of the background, stream, and off-track models. The mixture weights are learned from the data. Note that we use the `tied_coordinates` argument to specify that the off-track model "pm1" ($\mu_1$) coordinate should be "tied" to the stream model:

In [ ]:
# mm = ComponentMixtureModel(
#     dist.Dirichlet(jnp.ones(3)),
#     components=[bkg_model, stream_model, offtrack_model],
#     tied_coordinates={"offtrack": {"pm1": "stream"}},
# )

# mm = ComponentMixtureModel(
#     dist.Dirichlet(jnp.array([0.25, 0.25, 1.0, 1.0])),
#     components=[bkg_model, stream_model, offtrack_model_dense, offtrack_model_sparse],
#     tied_coordinates={"offtrack_dense": {"pm1": "stream"}, "offtrack_sparse": {"pm1": "stream"}},
# )
mm = ComponentMixtureModel(
    dist.Dirichlet(jnp.array([0.25, 0.25, 1.0])),
    components=[bkg_model, stream_model, offtrack_model_dense],
    tied_coordinates={"offtrack_dense": {"pm1": "stream"}},
)

We now try evaluating the numpyro model at a random sample of the parameter space to make sure it runs without errors:

In [ ]:
with numpyro.validation_enabled():
    seed(mm, jax.random.PRNGKey(42))(data, err)

Success! Now we can run an optimizer to find the MAP parameters. There are a few ways to do this with a numpyro model, but we will use the SVI machinery with a delta-function "guide" distribution, because later we will use SVI to generate posterior samples from the model. To do the optimization, we here use the Adam optimizer because we start from a random sample from the prior. In a real application, we would start from a good guess of the MAP parameters and could then use an optimizer like BFGS.

In [ ]:
optimizer = numpyro.optim.Adam(1e-1)
init_params = mm.pack_params(
    {
        "background": bkg_init_params,
        "stream": stream_init_params,
        # "offtrack": offtrack_init_params,
        "offtrack_dense": offtrack_init_params_dense,
        # "offtrack_sparse": offtrack_init_params_sparse,
    }
)
guide = AutoDelta(mm, init_loc_fn=numpyro.infer.init_to_value(values=init_params))
MAP_svi = SVI(mm, guide, optimizer, Trace_ELBO())

In [ ]:
rng_key = jax.random.PRNGKey(12345)
# init_state = MAP_svi.init(rng_key, data=data)
# init_params = MAP_svi.get_params(init_state)

thing = Predictive(guide, params=init_params, num_samples=1)
MAP_p0 = thing(rng_key, data=data)
MAP_p0 = {k: v[0] for k, v in MAP_p0.items()}

In [ ]:
mm.plot_model_projections(
    MAP_p0, plot_grids, pcolormesh_kwargs=dict(norm=mpl.colors.LogNorm())
)

In [ ]:
rng_key = jax.random.PRNGKey(54321)
keys = jax.random.split(rng_key, num=3)

MAP_svi_results = MAP_svi.run(keys[0], 1_000, data=data)
plt.loglog(MAP_svi_results.losses)

thing = Predictive(guide, params=MAP_svi_results.params, num_samples=1)
MAP_p = thing(keys[1], data=data)
MAP_p = {k: v[0] for k, v in MAP_p.items()}

In [ ]:
mm.plot_model_projections(MAP_p, plot_grids)
plot_data_projections(data, plot_grids)

In [ ]:
expanded_pars = mm.expand_numpyro_params(MAP_p)
for name in mm.component_names:
    mm._components[name].plot_model_projections(expanded_pars[name], plot_grids)

TODO: how to init guide at loc's given by MAP optimization?

In [ ]:
optimizer = numpyro.optim.Adam(1e-2)
guide = AutoNormal(mm, init_loc_fn=numpyro.infer.init_to_value(values=MAP_p))
svi = SVI(mm, guide, optimizer, Trace_ELBO())
svi_results = svi.run(jax.random.PRNGKey(0), 10_000, data=data)
plt.loglog(svi_results.losses)

In [ ]:
rng_key = jax.random.PRNGKey(123)

thing = Predictive(guide, params=svi_results.params, num_samples=100)
SVI_ps = thing(rng_key, data=data)
SVI_p = {k: v[0] for k, v in SVI_ps.items()}

In [ ]:
SVI_p

In [ ]:
mm.plot_model_projections(SVI_p, plot_grids)

In [ ]:
plt.hist(err["pm1"], bins=128, density=True)
plt.hist(np.exp(SVI_ps["stream:pm1:ln_scale_vals"]), bins=16, density=True)

In [ ]:
grids = {
    "phi1": jnp.linspace(*phi1_lim, 512),
    "phi2": jnp.linspace(*phi2_lim, 128),
    "pm1": jnp.linspace(-15, pm1_lim[1], 128),
}
mm.plot_model_projections(SVI_p, grids)
plot_data_projections(data, grids)

In [ ]:
rng_key = jax.random.PRNGKey(123987)

from numpyro.infer.util import unconstrain_fn

mcmc_p0 = unconstrain_fn(mm, (data,), {}, MAP_p)

# , init_strategy=numpyro.infer.init_to_value(values=MAP_p))
nuts_kernel = NUTS(mm, max_tree_depth=(5, 10))
mcmc = MCMC(nuts_kernel, num_warmup=1000, num_samples=100)
mcmc.run(rng_key, data=data, init_params=mcmc_p0)

In [ ]:
idata = az.from_numpyro(mcmc)
az.summary(idata.posterior)

samples = mcmc.get_samples()

In [ ]:
plt.scatter(
    stream_phi2_knots, true_phi2_loc_spl(stream_phi2_knots), marker="o", color="r"
)
plt.errorbar(
    stream_phi2_knots,
    idata.posterior["stream:phi2:loc_vals"][0].mean(axis=0),
    idata.posterior["stream:phi2:loc_vals"][0].std(axis=0),
)

In [ ]:
plt.hist(np.exp(idata.posterior["stream:pm1:ln_scale_vals"][0]), bins=32)
plt.axvline(0.05)

In [ ]:
# mean_probs = idata.posterior["bkg-grid-probs"][0].mean(axis=0)
plt.hist(idata.posterior["mixture-probs"][0, :, 0], bins=32)

Old debugging

In [ ]:
model = mm
model_args = ()
model_kwargs = {"data": data}
init_strategy = numpyro.infer.init_to_sample()
key, subkey = jax.random.split(jax.random.PRNGKey(42))

from functools import partial

from numpyro.distributions.transforms import biject_to
from numpyro.handlers import seed, substitute, trace
from numpyro.infer.util import _unconstrain_reparam, log_density, transform_fn

# Wrap model in a `substitute` handler to initialize from `init_loc_fn`.
model_ = model
seeded_model = substitute(seed(model_, subkey), substitute_fn=init_strategy)
model_trace = trace(seeded_model).get_trace(*model_args, **model_kwargs)
constrained_values, inv_transforms = {}, {}
for k, v in model_trace.items():
    if (
        v["type"] == "sample"
        and not v["is_observed"]
        and not v["fn"].support.is_discrete
    ):
        constrained_values[k] = v["value"]
        inv_transforms[k] = biject_to(v["fn"].support)
params = transform_fn(
    inv_transforms,
    {k: v for k, v in constrained_values.items()},
    invert=True,
)
print(list(params.keys()))

substituted_model = substitute(
    model, substitute_fn=partial(_unconstrain_reparam, params)
)
log_joint, model_trace = log_density(substituted_model, model_args, model_kwargs, {})